# PyNastran - Buckling Analysis

The iPython notebook for this demo can be found in:
   - docs\quick_start\demo\op2_pandas_multi_case.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/op2_pandas_multi_case.ipynb

In [1]:
import sys
import os
LIBRARIES = ['../../../', '../../../../humanfriendly', '../../../../tabulate']
for lib in LIBRARIES:
    if not os.path.exists(lib) :
        print('The specified libray path does not exists.')
        sys.exit(1)
    if os.path.abspath(lib) not in sys.path:
        sys.path.append(os.path.abspath(lib))
import humanfriendly
#import tabulate
from tabulate import tabulate
import pyNastran

In [2]:
import glob

import numpy as np
np.set_printoptions(formatter={'all':lambda x: '%g'})

import pandas as pd
# Sets default precision of real numbers for pandas output\n"
pd.set_option('precision', 2)

from IPython.core.display import display, HTML

from pyNastran.op2.op2 import read_op2

pkg_path = pyNastran.__path__[0]
model_path = os.path.join(pkg_path, '..', 'models')

In [4]:
OP2_fname = glob.glob(os.path.join('..', '..', '..', 'data', '*buckling*.op2'))[1]
#Returns real size of file in bytes
print('Size of the example file is %s.' %humanfriendly.format_size(os.path.getsize(OP2_fname), binary=True))
#print(humanfriendly.format_size(os.stat(OP2_fname).st_size, binary=True))

Size of the example file is 178.57 KiB.


In [5]:
#read_op2?
#dir(tabulate)

Let's show off ``combine=True/False``.  We'll talk about the keys soon.

In [6]:
DISPLACEMENTS_HEADER = ['ID','SUB ID', 'SUB SUB ID', 'Description']

In [7]:
#%%timeit
result = read_op2(OP2_fname, combine=True, debug=False, build_dataframe=True)

INFO:    op2_scalar.py:1173           op2_filename = '../../../data/buckling_solid_shell_bar.op2'


In [8]:
print(result.get_op2_stats(short=True))

displacements[(1, 1, 1, 0, 'DEFAULT1')]
eigenvectors[(1, 8, 1, 0, 'DEFAULT1')]
spc_forces[(1, 1, 1, 0, 'DEFAULT1')]
spc_forces[(1, 8, 1, 0, 'DEFAULT1')]
mpc_forces[(1, 1, 1, 0, 'DEFAULT1')]
grid_point_forces[(1, 1, 1, 0, 'DEFAULT1')]
grid_point_forces[(1, 2, 1, 0, 'DEFAULT1')]
load_vectors[(1, 1, 1, 0, 'DEFAULT1')]
crod_stress[(1, 1, 1, 0, 'DEFAULT1')]
crod_stress[(1, 8, 1, 0, 'DEFAULT1')]
crod_strain[(1, 1, 1, 0, 'DEFAULT1')]
crod_strain[(1, 8, 1, 0, 'DEFAULT1')]
cbar_stress[(1, 1, 1, 0, 'DEFAULT1')]
cbar_stress[(1, 8, 1, 0, 'DEFAULT1')]
cbar_strain[(1, 1, 1, 0, 'DEFAULT1')]
cbar_strain[(1, 8, 1, 0, 'DEFAULT1')]
cbar_force[(1, 1, 1, 0, 'DEFAULT1')]
cbar_force[(1, 8, 1, 0, 'DEFAULT1')]
cbeam_stress[(1, 1, 1, 0, 'DEFAULT1')]
cbeam_stress[(1, 8, 1, 0, 'DEFAULT1')]
cbeam_strain[(1, 1, 1, 0, 'DEFAULT1')]
cbeam_strain[(1, 8, 1, 0, 'DEFAULT1')]
cbeam_force[(1, 1, 1, 0, 'DEFAULT1')]
cbeam_force[(1, 8, 1, 0, 'DEFAULT1')]
ctria3_stress[(1, 1, 1, 0, 'DEFAULT1')]
ctria3_stress[(1, 8, 1, 0, 'DEFAU

In [ ]:
display(HTML(tabulate.tabulate(result.displacements.keys(), headers=DISPLACEMENTS_HEADER, tablefmt='html')))

In [ ]:
display(HTML(tabulate.tabulate(result.spc_forces.keys(), headers=DISPLACEMENTS_HEADER, tablefmt='html')))

In [ ]:
display(HTML(tabulate.tabulate(result.load_vectors.keys(), headers=DISPLACEMENTS_HEADER, tablefmt='html')))

In [9]:
display(HTML(tabulate.tabulate(result.cquad4_stress.keys(), headers=DISPLACEMENTS_HEADER, tablefmt='html')))

,ID,SUB ID,SUB SUB ID,Description
1,1,1,0,DEFAULT1
1,8,1,0,DEFAULT1


## Single Subcase Buckling Example

The keys cannot be "combined" despite us telling the program that it was OK.
We'll get the following values that we need to handle.
#### isubcase, analysis_code, sort_method, count, subtitle
 * isubcase -> the same key that you're used to accessing
 * sort_method -> 1 (SORT1), 2 (SORT2)
 * count -> the optimization count
 * subtitle -> the analysis subtitle (changes for superlements)
 * analysis code -> the "type" of solution 

 ### Partial code for calculating analysis code:
 
       if trans_word == 'LOAD STEP':  # nonlinear statics
          analysis_code = 10
      elif trans_word in ['TIME', 'TIME STEP']:  # TODO check name
          analysis_code = 6
      elif trans_word == 'EIGENVALUE':  # normal modes
          analysis_code = 2
      elif trans_word == 'FREQ':  # TODO check name
          analysis_code = 5
      elif trans_word == 'FREQUENCY':
          analysis_code = 5
      elif trans_word == 'COMPLEX EIGENVALUE':
          analysis_code = 9
      else:
          raise NotImplementedError('transient_word=%r is not supported...' % trans_word)

 

### Let's look at an odd case:

You can do buckling as one subcase or two subcases (makes parsing it a lot easier!).

However, you **have** to do this once you start messing around with superelements or multi-step optimization.

For optimization, sometimes Nastran will downselect elements and do an optimization on that and print out a subset of the elements.
At the end, it will rerun an analysis to double check the constraints are satisfied.
It does not always do multi-step optimization.

In [23]:
stress_keys = list(result.cquad4_stress.keys())
print(stress_keys)

[(1, 1, 1, 0, 'DEFAULT1'), (1, 8, 1, 0, 'DEFAULT1')]


Similarly:
 * Transient solutions can have preload
 * Frequency solutions can have loadsets (???)

## Moving onto the data frames
 * The static case is the initial deflection state
 * The buckling case is "transient", where the modes (called load steps or lsdvmn here) represent the "times"
 
pyNastran reads these tables differently and handles them differently internally.  They look very similar though.

In [18]:
stress_static = result.cquad4_stress[stress_keys[0]].data_frame
stress_transient = result.cquad4_stress[stress_keys[1]].data_frame

# The final calculated factor:
#   Is it a None or not?
# This defines if it's static or transient
print('stress_static.nonlinear_factor = %s' % result.cquad4_stress[key0].nonlinear_factor)
print('stress_transient.nonlinear_factor = %s' % result.cquad4_stress[key1].nonlinear_factor)

print('data_names  = %s' % result.cquad4_stress[key1].data_names)
print('loadsteps   = %s' % result.cquad4_stress[key1].lsdvmns)
print('eigenvalues = %s' % result.cquad4_stress[key1].eigrs)

stress_static.nonlinear_factor = None
stress_transient.nonlinear_factor = 4
data_names  = ['lsdvmn', 'eigr']
loadsteps   = [1, 2, 3, 4]
eigenvalues = [-49357660160.0, -58001940480.0, -379750744064.0, -428462538752.0]


## Static Table

In [21]:
stress_static.head(10)

index  fiber_distance       oxx       oyy  \
ElementID NodeID Location                                              
6         CEN    Top           0           -0.12  5.85e-07  9.73e-06   
                 Bottom        1            0.12  4.71e-07  9.44e-06   
          4      Top           2           -0.12 -6.50e-07  9.48e-06   
                 Bottom        3            0.12 -8.37e-07  9.11e-06   
          1      Top           4           -0.12 -6.50e-07  9.98e-06   
                 Bottom        5            0.12 -8.37e-07  9.76e-06   
          14     Top           6           -0.12  1.82e-06  9.98e-06   
                 Bottom        7            0.12  1.78e-06  9.76e-06   
          15     Top           8           -0.12  1.82e-06  9.48e-06   
                 Bottom        9            0.12  1.78e-06  9.11e-06   

                                txy  angle      omax      omin  von_mises  
ElementID NodeID Location                                                  
6         CEN    Top      -1.36e-07 -89.15  9.73e-06  5.83e-07   9.46e-06  
                 Bottom   -1.61e-07 -88.97  9.44e-06  4.69e-07   9.21e-06  
          4      Top      -1.36e-07 -89.23  9.48e-06 -6.52e-07   9.82e-06  
                 Bottom   -1.61e-07 -89.08  9.12e-06 -8.39e-07   9.56e-06  
          1      Top      -1.36e-07 -89.27  9.99e-06 -6.51e-07   1.03e-05  
                 Bottom   -1.61e-07 -89.13  9.76e-06 -8.39e-07   1.02e-05  
          14     Top      -1.36e-07 -89.05  9.99e-06  1.82e-06   9.21e-06  
                 Bottom   -1.61e-07 -88.85  9.76e-06  1.78e-06   9.01e-06  
          15     Top      -1.36e-07 -88.98  9.48e-06  1.82e-06   8.72e-06  
                 Bottom   -1.61e-07 -88.75  9.12e-06  1.78e-06   8.37e-06

## Transient Table

In [22]:
stress_transient.head(10)

LoadStep                             Item              1              2  \
EigenvalueReal                            -49357660160.0 -58001940480.0   
ElementID NodeID Location                                                 
6         CEN    Top       fiber_distance          -0.12      -1.25e-01   
                 Top                  oxx      -36570.46      -1.59e+05   
                 Top                  oyy      206374.97       1.08e+06   
                 Top                  txy         229.65      -1.27e+04   
                 Top                angle          89.95      -8.94e+01   
                 Top                 omax      206375.19       1.08e+06   
                 Top                 omin      -36570.67      -1.59e+05   
                 Top            von_mises      226881.94       1.17e+06   
                 Bottom    fiber_distance           0.12       1.25e-01   
                 Bottom               oxx      -28156.80      -9.56e+04   

LoadStep                                3               4  
EigenvalueReal            -379750744064.0 -428462538752.0  
ElementID NodeID Location                                  
6         CEN    Top            -1.25e-01       -1.25e-01  
                 Top            -1.50e+05        1.07e+06  
                 Top             4.03e+05        6.16e+06  
                 Top             4.39e+06       -3.57e+05  
                 Top             4.68e+01       -8.60e+01  
                 Top             4.53e+06        6.18e+06  
                 Top            -4.28e+06        1.04e+06  
                 Top             7.63e+06        5.73e+06  
                 Bottom          1.25e-01        1.25e-01  
                 Bottom         -1.94e+05       -4.88e+05